In [7]:
# ----------------- EDIT ONLY THE PATHS BELOW -----------------
PHASE1_PATH = r"D:\SolarTiltProject\data\data_150ohm.csv"        # <-- path to Phase I 150Ω CSV (GBK)
BLOCKS_SUM = r"D:\SolarTiltProject\data\20-200ohm\blocks_summary.csv" # <-- path to blocks_summary.csv (GBK)
RAW_SWEEP = r"D:\SolarTiltProject\data\data_20-200ohm.csv"  # <-- optional raw sweep CSV (GBK) or "" if none
OUT_DIR = r"D:\SolarTiltProject\data\mismatch"                      # <-- output dir for CSV + plots
ENCODING = "gbk"   # <- file encoding, e.g., "gbk" or "utf-8"
RESAMPLE_FREQ = None   # <- e.g. '1T' to resample Phase I to 1-minute cadence (or None)
VERBOSE = True
# ==============================================

import warnings, math
warnings.filterwarnings("ignore")
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

out_dir = Path(OUT_DIR)
out_dir.mkdir(parents=True, exist_ok=True)

def vlog(*a, **kw):
    if VERBOSE:
        print(*a, **kw)

# ----- helper: encoding-safe read -----
def read_csv_try_enc(path, enc_first=ENCODING, **kwargs):
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"File not found: {p}")
    tried = []
    enc_list = [enc_first, 'utf-8', 'utf-8-sig', 'cp1252', 'latin-1']
    for e in enc_list:
        try:
            df = pd.read_csv(p, encoding=e, engine='python', dtype=str, **kwargs)
            vlog(f"Read {p.name} with encoding='{e}'")
            return df
        except Exception as exc:
            tried.append((e,str(exc)))
    # final fallback
    df = pd.read_csv(p, engine='python', dtype=str, **kwargs)
    vlog(f"Read {p.name} with pandas default engine (fallback)")
    return df

# ----- column detection helpers -----
def find_datetime_col(df):
    for c in df.columns:
        lc = c.lower()
        if 'time' in lc or 'date' in lc or 'timestamp' in lc:
            return c
    # fallback: first parseable
    for c in df.columns:
        try:
            parsed = pd.to_datetime(df[c].dropna().astype(str).iloc[:8], errors='coerce')
            if parsed.notna().any():
                return c
        except Exception:
            pass
    return None

def find_power_col(df):
    prefs = ['pmax','p_max','pmax_w','pmax_w','p_max_w','mean_power_w','mean_power','power_w','power']
    for p in prefs:
        for c in df.columns:
            if c.lower().replace(' ','_') == p:
                return c
    # heuristic: any column name containing 'power' or 'pmax'
    for c in df.columns:
        if 'power' in c.lower() or 'pmax' in c.lower():
            return c
    return None

def find_voltage_current_cols(df):
    v = None; i = None
    for c in df.columns:
        lc = c.lower()
        if 'volt' in lc or lc == 'v':
            if v is None: v = c
        if 'current' in lc or 'amp' in lc or lc == 'i':
            if i is None: i = c
    return v, i

# ----- Phase II: determine MPP and Ropt (prefer blocks_summary, else raw sweep) -----
bs_df = read_csv_try_enc(BLOCKS_SUM, skip_blank_lines=False)
vlog("blocks_summary columns:", bs_df.columns.tolist())

# try to obtain Pmax and Ropt from blocks_summary (if you already marked them)
Pmax_from_bs = None
Vmp_from_bs = None
Imp_from_bs = None
Ropt_from_bs = None

# common candidate names
candidate_names = {
    'pmax': ['pmax','p_max','pmax_w','p_max_w','pmax_watt','pmax_watts','Pmax','P_max','Pmax_W'],
    'vmp': ['vmp','v_mp','vmp_v','Vmp','V_mp','Vmp_V'],
    'imp': ['imp','i_mp','imp_a','Imp','I_mp','Imp_A'],
    'ropt':['r_opt','r_at_mpp','r_opt_ohm','Ropt','R_opt','R_at_mpp','R']
}

# lower-cased keys for matching
cols_lc = {c.lower():c for c in bs_df.columns}

# helper to find any of a list in columns (case-insensitive)
def find_col_by_list(dfcols, names_list):
    for nm in names_list:
        for c in dfcols:
            if c.lower().replace(' ','_') == nm.lower().replace(' ','_'):
                return c
    # fallback fuzzy contains
    for nm in names_list:
        for c in dfcols:
            if nm.lower().replace('_','') in c.lower().replace('_',''):
                return c
    return None

# If blocks_summary has multiple rows, use the row that is marked or the max-power row
# We'll try to find the first row with numeric Pmax candidate; otherwise choose idxmax if numeric col exists
# 1) search for Pmax-like column name
pcol = find_power_col(bs_df)
if pcol is not None:
    # coerce numeric
    try:
        bs_df[pcol] = pd.to_numeric(bs_df[pcol], errors='coerce')
    except Exception:
        pass

# pick the representative block row:
rep_row = None
if bs_df.shape[0] == 1:
    rep_row = bs_df.iloc[0]
else:
    # if one column explicitly named 'Sweep_ID' or 'block' etc you might have chosen; otherwise pick the row with largest numeric power if present
    if pcol is not None and bs_df[pcol].notna().any():
        rep_idx = bs_df[pcol].astype(float).idxmax()
        rep_row = bs_df.loc[rep_idx]
    else:
        # default to first non-empty row
        rep_row = bs_df.iloc[0]

# attempt to extract values from the representative row
if rep_row is not None:
    # Pmax
    col_p = find_col_by_list(bs_df.columns, candidate_names['pmax'])
    if col_p is not None and str(rep_row.get(col_p)).strip() != '':
        try:
            Pmax_from_bs = float(str(rep_row[col_p]).strip())
        except Exception:
            Pmax_from_bs = None
    # Vmp
    col_v = find_col_by_list(bs_df.columns, candidate_names['vmp'])
    if col_v is not None and str(rep_row.get(col_v)).strip() != '':
        try:
            Vmp_from_bs = float(str(rep_row[col_v]).strip())
        except Exception:
            Vmp_from_bs = None
    # Imp
    col_i = find_col_by_list(bs_df.columns, candidate_names['imp'])
    if col_i is not None and str(rep_row.get(col_i)).strip() != '':
        try:
            Imp_from_bs = float(str(rep_row[col_i]).strip())
        except Exception:
            Imp_from_bs = None
    # Ropt
    col_r = find_col_by_list(bs_df.columns, candidate_names['ropt'])
    if col_r is not None and str(rep_row.get(col_r)).strip() != '':
        try:
            Ropt_from_bs = float(str(rep_row[col_r]).strip())
        except Exception:
            Ropt_from_bs = None

vlog("From blocks_summary (if present): Pmax=", Pmax_from_bs, "Vmp=", Vmp_from_bs, "Imp=", Imp_from_bs, "Ropt=", Ropt_from_bs)

# If necessary, compute MPP from RAW_SWEEP
Pmax_computed = None; Vmp_computed = None; Imp_computed = None; Ropt_computed = None
sweep_used = None

raw_path = Path(RAW_SWEEP)
if raw_path.exists():
    raw_df = read_csv_try_enc(RAW_SWEEP)
    tcol = find_datetime_col(raw_df)
    vcol, icol = find_voltage_current_cols(raw_df)
    pcol = find_power_col(raw_df)
    # coerce numeric where possible
    for c in raw_df.columns:
        try:
            raw_df[c] = pd.to_numeric(raw_df[c], errors='coerce')
        except Exception:
            pass
    # compute power if needed
    if pcol is None:
        if vcol is not None and icol is not None:
            # ensure current in A (if values >> 10 assume mA)
            if raw_df[icol].median(skipna=True) is not np.nan and raw_df[icol].median(skipna=True) > 10:
                raw_df[icol] = raw_df[icol] / 1000.0
            raw_df['Power_calc'] = raw_df[vcol] * raw_df[icol]
            pcol = 'Power_calc'
        elif vcol is not None:
            # maybe sweep with resistor recorded -> try V^2/R if R column exists
            rcol = next((c for c in raw_df.columns if 'resist' in c.lower() or 'ohm' in c.lower() or 'res'==c.lower()), None)
            if rcol is not None:
                raw_df['Power_calc'] = raw_df[vcol]**2 / pd.to_numeric(raw_df[rcol], errors='coerce').replace({0:np.nan})
                pcol = 'Power_calc'
    # find max-power row
    if pcol is not None and raw_df[pcol].notna().any():
        idx_max = raw_df[pcol].astype(float).idxmax()
        Pmax_computed = float(raw_df.loc[idx_max, pcol])
        if vcol is not None:
            Vmp_computed = float(raw_df.loc[idx_max, vcol])
        if icol is not None:
            Imp_computed = float(raw_df.loc[idx_max, icol])
        # if current was mA and we converted earlier, Imp_computed is in A
        if Vmp_computed is not None and Imp_computed is not None and Imp_computed != 0:
            Ropt_computed = Vmp_computed / Imp_computed
        # determine sweep timestamp if available
        if tcol is not None and pd.notna(raw_df.loc[idx_max, tcol]):
            sweep_used = pd.to_datetime(raw_df.loc[idx_max, tcol], errors='coerce')
    vlog("Computed from raw sweep: Pmax=", Pmax_computed, "Vmp=", Vmp_computed, "Imp=", Imp_computed, "Ropt=", Ropt_computed)

# FINAL: choose authoritative values (priority: blocks_summary values if present, else computed)
Pmax = Pmax_from_bs if Pmax_from_bs is not None else Pmax_computed
Vmp  = Vmp_from_bs  if Vmp_from_bs  is not None else Vmp_computed
Imp  = Imp_from_bs  if Imp_from_bs  is not None else Imp_computed
Ropt = Ropt_from_bs if Ropt_from_bs is not None else Ropt_computed

# If still missing Pmax, fail with clear message
if Pmax is None:
    raise RuntimeError("Pmax not found in blocks_summary and could not be computed from RAW_SWEEP. Please ensure one of them contains power values.")

vlog("Final authoritative MPP values being used:")
vlog(f"  Pmax = {Pmax:.5g} W")
if (Vmp is not None) and (Imp is not None):
    vlog(f"  Vmp = {Vmp:.5g} V, Imp = {Imp:.5g} A, Ropt = {Ropt:.5g} Ω")
else:
    vlog("  Note: Vmp/Imp not fully available; Ropt may be NaN")

# If both blocks_summary Pmax and computed Pmax exist and differ, warn and report both
if (Pmax_from_bs is not None) and (Pmax_computed is not None):
    rel_diff = abs(Pmax_from_bs - Pmax_computed) / max(abs(Pmax_computed),1e-12)
    if rel_diff > 0.01:
        vlog("WARNING: Pmax in blocks_summary differs from Pmax computed from RAW_SWEEP by {:.2%}".format(rel_diff))
        vlog("  blocks_summary Pmax =", Pmax_from_bs, "; computed Pmax =", Pmax_computed)

# ----- Now compute mismatch against Phase I 150Ω timeseries -----
# load Phase I and preprocess duplicates
phase1_df = read_csv_try_enc(PHASE1_PATH)
dtcol = find_datetime_col(phase1_df)
if dtcol is None:
    raise RuntimeError("Phase I file has no detectable datetime column.")
# parse and aggregate duplicates
phase1_df[dtcol] = pd.to_datetime(phase1_df[dtcol].astype(str), errors='coerce')
phase1_df = phase1_df.dropna(subset=[dtcol]).set_index(dtcol).sort_index()
# coerce numerics
for c in phase1_df.columns:
    try: phase1_df[c] = pd.to_numeric(phase1_df[c], errors='coerce')
    except: pass
# aggregate duplicate timestamps by mean
if not phase1_df.index.is_unique:
    phase1_df = phase1_df.groupby(phase1_df.index).mean()
# determine power column
pcol150 = find_power_col(phase1_df)
if pcol150 is None:
    vcol150, icol150 = find_voltage_current_cols(phase1_df)
    if (vcol150 is not None) and (icol150 is not None):
        # convert current to A if needed heuristically
        if phase1_df[icol150].median(skipna=True) is not np.nan and phase1_df[icol150].median(skipna=True) > 10:
            phase1_df[icol150] = phase1_df[icol150] / 1000.0
        phase1_df['Power_W'] = phase1_df[vcol150] * phase1_df[icol150]
        pcol150 = 'Power_W'
    elif vcol150 is not None:
        phase1_df['Power_W'] = (phase1_df[vcol150]**2) / 150.0
        pcol150 = 'Power_W'
    else:
        raise RuntimeError("Phase I file: cannot determine or compute Power column.")

phase1_df[pcol150] = pd.to_numeric(phase1_df[pcol150], errors='coerce')

# find nearest phase1 sample to sweep timestamp
# prefer sweep_used (from raw sweep), else try to use timestamp in blocks_summary row
sweep_ts = None
# try blocks_summary timestamp field
for cand in ['time_of_mpp','time','start_time','end_time','timestamp','datetime','date','day']:
    if cand in bs_df.columns and pd.notna(rep_row.get(cand)):
        try:
            sweep_ts = pd.to_datetime(str(rep_row.get(cand)), errors='coerce')
            if not pd.isna(sweep_ts):
                break
        except Exception:
            sweep_ts = None
# if not found, use sweep_used from computed raw sweep
if (sweep_ts is None or pd.isna(sweep_ts)) and (sweep_used is not None):
    sweep_ts = sweep_used
# final fallback: use timestamp of phase1 peak
if sweep_ts is None or pd.isna(sweep_ts):
    sweep_ts = phase1_df[pcol150].astype(float).idxmax()
vlog("Using sweep timestamp:", sweep_ts)

# robust nearest-sample lookup (works even if coarse timestamps)
time_diffs = (phase1_df.index.to_series() - pd.to_datetime(sweep_ts)).abs()
if time_diffs.empty:
    raise RuntimeError("Phase I has no valid timestamps after preprocessing.")
nearest_pos = int(np.argmin(time_diffs.values))
nearest_time = phase1_df.index[nearest_pos]
P150_at_sweep = float(phase1_df.iloc[nearest_pos][pcol150])
vlog(f"Nearest Phase I sample time: {nearest_time} -> Power @150Ω = {P150_at_sweep:.5g} W")

# instantaneous mismatch
instant_mismatch_pct = 100.0 * (Pmax - P150_at_sweep) / Pmax

# conservative daily energy loss
day = nearest_time.normalize()
mask = (phase1_df.index >= day) & (phase1_df.index < day + pd.Timedelta(days=1))
df_day = phase1_df.loc[mask]
if df_day.empty:
    E150_Wh = np.nan; Emax_Wh = np.nan; daily_loss_pct = np.nan
else:
    times = (df_day.index.view('int64')/1e9).astype(float)
    Pvals = df_day[pcol150].astype(float).values
    E150_J = np.trapz(Pvals, times) if len(Pvals) >= 2 else 0.0
    E150_Wh = E150_J / 3600.0
    Pmax_arr = np.full_like(Pvals, fill_value=Pmax, dtype=float)
    Emax_J = np.trapz(Pmax_arr, times) if len(Pmax_arr) >= 2 else Pmax * ( (times[-1]-times[0]) if len(times)>=2 else 0.0 )
    Emax_Wh = Emax_J / 3600.0
    daily_loss_pct = 100.0 * (Emax_Wh - E150_Wh) / Emax_Wh if Emax_Wh > 0 else np.nan

# Save summary and overlay plot
out_dict = {
    'sweep_timestamp_used': [str(sweep_ts)],
    'Pmax_sweep_W': [Pmax],
    'Vmp_used_V': [Vmp],
    'Imp_used_A': [Imp],
    'Ropt_used_Ohm': [Ropt],
    'P150_nearest_W': [P150_at_sweep],
    'instantaneous_mismatch_pct': [instant_mismatch_pct],
    'day': [str(day.date())],
    'E_150_Wh': [E150_Wh],
    'E_max_Wh': [Emax_Wh],
    'daily_loss_pct': [daily_loss_pct]
}
out_df = pd.DataFrame(out_dict)
csv_out = out_dir / "mismatch_summary.csv"
out_df.to_csv(csv_out, index=False, encoding='utf-8')
vlog("Saved mismatch_summary.csv ->", csv_out)

# overlay plot: Phase I power for that day + horizontal Pmax
if not df_day.empty:
    fig, ax = plt.subplots(figsize=(10,4))
    ax.plot(df_day.index, df_day[pcol150], marker='o', linewidth=1, label='Power At 150Ω (mW)')
    ax.axhline(Pmax, color='C1', linestyle='--', label=f'Pmax (sweep) = {Pmax:.3f} mW')
    ax.fill_between(df_day.index, df_day[pcol150], Pmax, where=(Pmax>df_day[pcol150]), color='C1', alpha=0.25)
    ax.set_xlabel('Time')
    ax.set_ylabel('Power (mW)')
    ax.set_title('Phase I power (150Ω) on sweep day — overlay with sweep Pmax')
    ax.legend()
    plt.tight_layout()
    img_out = out_dir / "mismatch_overlay.png"
    fig.savefig(img_out, dpi=300, bbox_inches='tight')
    vlog("Saved overlay image ->", img_out)
    plt.show()
else:
    vlog("No Phase I daytime data for the sweep day -> overlay not generated.")

# Final printed summary for your report — definitive values
print("\n===== Final Mismatch Summary (authoritative) =====")
print(f"Pmax used (W) ............: {Pmax:.5g}")
print(f"Vmp used (V) .............: {Vmp if Vmp is not None else 'N/A'}")
print(f"Imp used (A) .............: {Imp if Imp is not None else 'N/A'}")
print(f"Ropt used (Ohm) ..........: {Ropt if Ropt is not None else 'N/A'}")
print(f"P150 at nearest sample (W): {P150_at_sweep:.5g}  (time {nearest_time})")
print(f"Instantaneous mismatch (%) : {instant_mismatch_pct:.3f} %")
print(f"Conservative daily loss (%) : {daily_loss_pct if not pd.isna(daily_loss_pct) else 'N/A'}")
print("Saved CSV:", csv_out)
if not df_day.empty:
    print("Saved overlay image:", img_out)


Read blocks_summary.csv with encoding='gbk'
blocks_summary columns: ['Block_ID', 'Resistance_Ohm', 'n_rows', 'mean_voltage_V', 'mean_current_A', 'mean_power_W', 'std_power_W', 'mean_light_lux', 'mean_panel_temp_C']
From blocks_summary (if present): Pmax= None Vmp= None Imp= None Ropt= 25.0
Read data_20-200ohm.csv with encoding='gbk'
Computed from raw sweep: Pmax= 7772.5 Vmp= 17.029 Imp= nan Ropt= nan
Final authoritative MPP values being used:
  Pmax = 7772.5 W
  Vmp = 17.029 V, Imp = nan A, Ropt = 25 Ω
Read data_150ohm.csv with encoding='gbk'
Using sweep timestamp: 2025-08-17 11:10:00
Nearest Phase I sample time: 2025-08-17 11:10:00 -> Power @150Ω = 2632.5 W
Saved mismatch_summary.csv -> D:\SolarTiltProject\data\mismatch\mismatch_summary.csv
Saved overlay image -> D:\SolarTiltProject\data\mismatch\mismatch_overlay.png

===== Final Mismatch Summary (authoritative) =====
Pmax used (W) ............: 7772.5
Vmp used (V) .............: 17.029
Imp used (A) .............: nan
Ropt used (Ohm) 